Data -> [link text](https://www.kaggle.com/c/fake-news/data#)

Fake News Classifier using LSTM

In [3]:
import numpy as np
import pandas as pd

In [5]:
from google.colab import files
uploaded = files.upload()

Saving train.csv to train.csv


In [118]:
df=pd.read_csv('train.csv')
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [119]:
df.shape

(20800, 5)

In [120]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [121]:
df=df.dropna()

In [122]:
df.shape

(18285, 5)

In [123]:
dep=df.drop('label',axis=1)

In [124]:
dep

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [125]:
ind=df['label']

In [126]:
titles=dep['title']
titles

0        House Dem Aide: We Didn’t Even See Comey’s Let...
1        FLYNN: Hillary Clinton, Big Woman on Campus - ...
2                        Why the Truth Might Get You Fired
3        15 Civilians Killed In Single US Airstrike Hav...
4        Iranian woman jailed for fictional unpublished...
                               ...                        
20795    Rapper T.I.: Trump a ’Poster Child For White S...
20796    N.F.L. Playoffs: Schedule, Matchups and Odds -...
20797    Macy’s Is Said to Receive Takeover Approach by...
20798    NATO, Russia To Hold Parallel Exercises In Bal...
20799                            What Keeps the F-35 Alive
Name: title, Length: 18285, dtype: object

Text Preprocessing :- Remove Punctuation Stop Words,Lemmatization

In [127]:
import nltk
nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer
lem=WordNetLemmatizer()
from nltk.corpus import stopwords
nltk.download('wordnet')
nltk.download('omw-1.4')
import re

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [128]:
n=dep['index'].size
corpus=[]
type(titles[i])
for i in range(n):
  temp=""
  for j in titles[i]:
    if j not in stopwords.words('english'):
      j=j.lower()
      j=lem.lemmatize(j)
      temp+=j+' '
  # print(temp)
  # print(type(temp))
  
#   temp=' '.join(temp)   
  temp=re.sub('[^a-zA-Z]', ' ',temp)
  corpus.append(temp)
corpus




KeyError: ignored

One Hot Encoding

In [129]:
import tensorflow as tf
print(tf.__version__)

2.9.2


In [130]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [131]:
voc_size=5000
emb=[one_hot(sent,voc_size)for sent in corpus]

Padding

In [136]:
sent_length=20
emb=pad_sequences(emb,maxlen=sent_length,padding='pre')
emb

array([[   0,    0,    0, ..., 3196,  804, 1902],
       [   0,    0,    0, ..., 2208, 3098, 2435],
       [   0,    0,    0, ..., 2402, 3838, 2254],
       ...,
       [   0,    0,    0, ...,  154, 4150, 4920],
       [   0,    0,    0, ...,  681, 2483, 4457],
       [   0,    0,    0, ...,  587, 3675, 3589]], dtype=int32)

Vector and model

In [140]:
## Creating model
emb_vector_features=40 ##features representation
model=Sequential()
model.add(Embedding(voc_size,emb_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid')) # Here we have problem of binary classifier that's why we are using sigmoid actiation function
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


Traing model

In [142]:
X_final=np.array(emb)
y_final=np.array(ind)

In [143]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

Fitting Model

In [144]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 10s 41ms/step - loss: 0.3217 - accuracy: 0.8525 - val_loss: 0.1909 - val_accuracy: 0.9181
Epoch 2/10
192/192 [==============================] - 7s 35ms/step - loss: 0.1258 - accuracy: 0.9524 - val_loss: 0.1925 - val_accuracy: 0.9244
Epoch 3/10
192/192 [==============================] - 7s 35ms/step - loss: 0.0736 - accuracy: 0.9741 - val_loss: 0.2337 - val_accuracy: 0.9269
Epoch 4/10
192/192 [==============================] - 7s 34ms/step - loss: 0.0423 - accuracy: 0.9856 - val_loss: 0.3004 - val_accuracy: 0.9183
Epoch 5/10
192/192 [==============================] - 7s 37ms/step - loss: 0.0236 - accuracy: 0.9936 - val_loss: 0.3337 - val_accuracy: 0.9243
Epoch 6/10
192/192 [==============================] - 10s 55ms/step - loss: 0.0110 - accuracy: 0.9970 - val_loss: 0.3705 - val_accuracy: 0.9231
Epoch 7/10
192/192 [==============================] - 7s 38ms/step - loss: 0.0060 - accuracy: 0.9986 - val_loss: 0.4151 - val_accuracy: 0.91

Performance Matrix

In [145]:
y_pred=model.predict(X_test)

189/189 [==============================] - 2s 6ms/step


In [148]:
y_pred=np.where(y_pred > 0.6, 1,0) ##AUC ROC Curve

In [149]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)

array([[3114,  305],
       [ 198, 2418]])

In [150]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9166528583264292

In [151]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.94      0.91      0.93      3419
           1       0.89      0.92      0.91      2616

    accuracy                           0.92      6035
   macro avg       0.91      0.92      0.92      6035
weighted avg       0.92      0.92      0.92      6035



adding dropout

In [153]:
from tensorflow.keras.layers import Dropout
emb_vector_features=40 ##features representation
model=Sequential()
model.add(Embedding(voc_size,emb_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid')) # Here we have problem of binary classifier that's why we are using sigmoid actiation function
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 20, 40)            200000    
                                                                 
 dropout (Dropout)           (None, 20, 40)            0         
                                                                 
 lstm_1 (LSTM)               (None, 100)               56400     
                                                                 
 dropout_1 (Dropout)         (None, 100)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [154]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 11s 40ms/step - loss: 0.3307 - accuracy: 0.8379 - val_loss: 0.2041 - val_accuracy: 0.9112
Epoch 2/10
192/192 [==============================] - 7s 37ms/step - loss: 0.1397 - accuracy: 0.9456 - val_loss: 0.1846 - val_accuracy: 0.9228
Epoch 3/10
192/192 [==============================] - 9s 49ms/step - loss: 0.1043 - accuracy: 0.9629 - val_loss: 0.2142 - val_accuracy: 0.9241
Epoch 4/10
192/192 [==============================] - 8s 40ms/step - loss: 0.0744 - accuracy: 0.9731 - val_loss: 0.2212 - val_accuracy: 0.9201
Epoch 5/10
192/192 [==============================] - 8s 44ms/step - loss: 0.0543 - accuracy: 0.9818 - val_loss: 0.2464 - val_accuracy: 0.9220
Epoch 6/10
192/192 [==============================] - 8s 43ms/step - loss: 0.0384 - accuracy: 0.9878 - val_loss: 0.2976 - val_accuracy: 0.9148
Epoch 7/10
192/192 [==============================] - 9s 45ms/step - loss: 0.0280 - accuracy: 0.9917 - val_loss: 0.3278 - val_accuracy: 0.916

In [155]:
y_pred=model.predict(X_test)

189/189 [==============================] - 2s 7ms/step


In [156]:
y_pred=np.where(y_pred > 0.6, 1,0)

In [157]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)

array([[3167,  252],
       [ 269, 2347]])

In [158]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9136702568351284

In [159]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.92      0.93      0.92      3419
           1       0.90      0.90      0.90      2616

    accuracy                           0.91      6035
   macro avg       0.91      0.91      0.91      6035
weighted avg       0.91      0.91      0.91      6035

